In [14]:
import datetime
import torch
from quantitative_analytics.indices import indices, indexfixingrepository
from quantitative_analytics.marketdata import marketdata, marketdatarepository
from quantitative_analytics.products import products
from quantitative_analytics.models import models
from quantitative_analytics.calculators import calculators

In [15]:
torch.set_printoptions(precision=16)

EPSILON = 0.00000001

1. Define some indices

In [16]:
equity = indices.EquityIndex([], "SPX")
equity2 = indices.EquityIndex([], "AAP")

2. Define the option data

In [17]:
observationDates = [datetime.date(year=2020, month=11, day=30),
                    datetime.date(year=2021, month=6, day=30),
                    datetime.date(year=2021, month=12, day=30)]
expiry = datetime.date(year=2021, month=12, day=30)

option_data = {}
option_data['strike'] = torch.tensor(100.0, requires_grad=True)
option_data['expiry'] = expiry
option_data['index'] = equity

# Create a European Option
europeanOption = products.EuropeanOptionProduct(option_data)

option_data['observationDates'] = observationDates

# Create an Asian Option
asianOption = products.AsianOptionProduct(option_data)

option_data['indices'] = [equity, equity2]
asianBasketOption = products.AsianBasketOptionProduct(option_data)

3. Define the model

In [18]:
modelData = {}

modelDate = datetime.date(year=2020, month=12, day=30)
model = models.LognormalModel(modelData, modelDate)

4. Define fixings 

In [19]:
indexfixingrepository.indexFixingRepositorySingleton.clear()

fixingDate = datetime.date(year=2020, month=11, day=30)
spot_fixing = torch.tensor(100.0, requires_grad=True)
indexfixingrepository.indexFixingRepositorySingleton.storeFixing(equity, fixingDate, spot_fixing)

spot_fixing2 = torch.tensor(100.0, requires_grad=True)
indexfixingrepository.indexFixingRepositorySingleton.storeFixing(equity2, fixingDate, spot_fixing2)

5. Define market data

In [20]:
marketdatarepository.marketDataRepositorySingleton.clear()

observationDates = [datetime.date(year=2020, month=11, day=30),
                    datetime.date(year=2021, month=6, day=30),
                    datetime.date(year=2021, month=12, day=30)]
expiry = datetime.date(year=2021, month=12, day=30)

forward = torch.tensor([100.0], requires_grad=True)
md = marketdata.MarketDataEquitySpotBase(equity, forward)
marketdatarepository.marketDataRepositorySingleton.storeMarketData(md)

forward2 = torch.tensor([100.0], requires_grad=True)
md2 = marketdata.MarketDataEquitySpotBase(equity2, forward2)
marketdatarepository.marketDataRepositorySingleton.storeMarketData(md2)

volatilityDates = [datetime.date(year=2021, month=6, day=30),
                    datetime.date(year=2021, month=12, day=30)]
volatilityPoint1 = torch.tensor([0.2], requires_grad=True)
volatilityPoint2 = torch.tensor([0.2], requires_grad=True)

volatilityValues = [volatilityPoint1, volatilityPoint2]
volatilityMarketData = marketdata.BlackVolatilityMarketData(equity, volatilityDates, volatilityValues)
marketdatarepository.marketDataRepositorySingleton.storeMarketData(volatilityMarketData)

volatilityPoint3 = torch.tensor([0.3], requires_grad=True)
volatilityPoint4 = torch.tensor([0.3], requires_grad=True)

volatilityValues2 = [volatilityPoint3, volatilityPoint4]
volatilityMarketData2 = marketdata.BlackVolatilityMarketData(equity2, volatilityDates, volatilityValues2)
marketdatarepository.marketDataRepositorySingleton.storeMarketData(volatilityMarketData2)

correlation = torch.tensor([0.1], requires_grad=True)
correlationMarketData = marketdata.CorrelationMarketData(equity2, equity, correlation)
marketdatarepository.marketDataRepositorySingleton.storeMarketData(correlationMarketData)

correlation2 = torch.tensor([0.1], requires_grad=True)
correlationMarketData2 = marketdata.CorrelationMarketData(equity, equity2, correlation2)
marketdatarepository.marketDataRepositorySingleton.storeMarketData(correlationMarketData2)




6. Define the simulation data and run

In [21]:
# Run the Monte-Carlo
simulationData = {}
simulationData['NumberOfSimulations'] = 100000

simulationData['LegValues'] = True
mc = calculators.MonteCarloSimulator(simulationData, model, europeanOption)
npvmc = mc.npv()

# Compute first order derivatives
x = [forward]

dxs = []
for it in npvmc:
    dx, = torch.autograd.grad(it, x, create_graph=True, retain_graph=True)
    dxs.append(dx)

ddxs = []
for it in dxs:
    ddx, = torch.autograd.grad(it, x, create_graph=True)
    ddxs.append(ddx)

print(npvmc)
print(dxs)
print(ddxs)

SPX
1
tensor([100.0324172973632812,   8.0145454406738281], grad_fn=<StackBackward>)
[tensor([1.0003243684768677], grad_fn=<SumBackward1>), tensor([0.5393523573875427], grad_fn=<SumBackward1>)]
[tensor([0.], grad_fn=<SumBackward1>), tensor([0.0141420736908913], grad_fn=<SumBackward1>)]
